In [22]:
import pandas as pd
import numpy as np
import time
import os 
from pandas import Series, DataFrame
from tqdm.notebook import tqdm

dir_ = '../data/'
file_name = 'filter_track_5_user_100.pkl'

In [23]:
lh = pd.read_pickle(os.path.join(dir_, file_name))

In [24]:
unique_users = lh['uid'].unique()
unique_user = len(unique_users)
unique_tracks = lh['tid'].unique()
unique_track = len(unique_tracks)

print(unique_user)
print(unique_track)

953
157567


In [25]:
lh[:5]

,uid,tid,count
10,0,10,27
17,0,17,7
18,0,18,8
20,0,20,8
21,0,21,7


In [26]:
track_keys = {}

for track in unique_tracks:
    track_keys[track] = len(track_keys)
    
user_keys = {}

for user in unique_users:
    user_keys[user] = len(user_keys)

In [27]:
lh['tid']= lh['tid'].map(track_keys)
lh['uid']= lh['uid'].map(user_keys)

In [28]:
lh[:5]

,uid,tid,count
10,0,0,27
17,0,1,7
18,0,2,8
20,0,3,8
21,0,4,7


In [29]:
threshold = []
for i in tqdm(lh['uid'].unique()):
    user = lh[lh['uid'] == i]
    user = user.sort_values(by=['count'])
    t = []
    for j in range(19):
        v = user.iloc[len(user)//20*(j+1)]['count']
        t.append(v)
    threshold.append(t)
threshold[0]

[1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 9, 10, 13, 19]

In [30]:
a = threshold[0][18]
print(a)

19


In [31]:
start = time.time()
def to_rating(df):  
    rating = 0
    for i in range(18,-1,-1):
        if df['count'] <= threshold[df['uid']][i]:
            continue
        else:
            rating = i+1
            break
    return 0.25*(rating+1)
        
lh['rating'] = lh.apply(to_rating , axis = 1)
lh = lh[['uid','tid','rating','count']]

end = time.time()
print(end - start)

238.17645382881165


In [32]:
lh[:5]

,uid,tid,rating,count
10,0,0,5.00,27
17,0,1,3.75,7
18,0,2,4.00,8
20,0,3,4.00,8
21,0,4,3.75,7


In [33]:
lh.to_pickle(os.path.join(dir_, 'normalized_to_rating_' + file_name))

In [34]:
# minmax

In [35]:
lh = pd.read_pickle(os.path.join(dir_, 'normalized_to_rating_' + file_name))
lh[:10]

,uid,tid,rating,count
10,0,0,5.00,27
17,0,1,3.75,7
18,0,2,4.00,8
20,0,3,4.00,8
21,0,4,3.75,7
22,0,5,3.50,6
23,0,6,3.50,6
24,0,7,4.00,9
25,0,8,4.75,17
26,0,9,4.75,15


In [36]:
max_list = {}
for i in tqdm(lh['uid'].unique()):
    user = lh[lh['uid'] == i]
    max_value = max(user['count'])
    max_list[i] = max_value

In [37]:
# If the input data is array, try the code below

# for i in train['uid'].unique():
#     user = train[train['uid'] == i]
#     max_value = max(user['count'])
#     print(max_value)
#     normalized_list[i] = [j *5 / max_value for j in counting_list[i]]

# for i in tqdm_notebook(lh['uid'].unique()):
#     user = lh[lh['uid'] == i]
#     max_value = max(user['count'])
#     for index, row in user.iterrows():
#         lh.loc[index, 'count'] = row['count'] *5 / max_value
start = time.time()
def normalize(df):  
    return int(df['count']) *5 / max_list[df['uid']]

lh['minmax'] = lh.apply(normalize , axis = 1)
lh = lh[['uid', 'tid', 'rating', 'minmax', 'count']]

end = time.time()
print(end - start)

42.017565965652466


In [38]:
lh[:5]

,uid,tid,rating,minmax,count
10,0,0,5.00,1.928571,27
17,0,1,3.75,0.500000,7
18,0,2,4.00,0.571429,8
20,0,3,4.00,0.571429,8
21,0,4,3.75,0.500000,7


In [39]:
lh.to_pickle(os.path.join(dir_, 'normalized_to_rating_' + file_name))